In [4]:
################ Code from the 'Optimization of molecular geometries' tutorial##################
# imports and relevant defines:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
import time

# Simulation starting parameters:
symbols = ["Be", "H", "H"]

# this is selected from the tutorial 'Building molecular Hamiltonians', which has the same nulcear-coordinats for water
x = np.array([ 0.8765    ,  1.9857    , -2.4895    ,  0.83258936,  0.27791693,
        -2.71614899,  0.9293593 ,  3.70744234, -2.5407848 ], requires_grad=True)

# these parameters are used to match up with the parameters for the VQE run on water in the paper: https://arxiv.org/pdf/2106.13840.pdf
active_electrons = 4
active_orbitals = 6

# define the hamiltonian needed to compute cost-function
def H(x):
    return qml.qchem.molecular_hamiltonian(symbols, x, charge=0, active_electrons = active_electrons, active_orbitals = active_orbitals)[0]



################ Code from the 'Building the adaptive circuit' tutorial below:################

geometry = np.array([ 0.8765    ,  1.9857    , -2.4895    ,  0.83258936,  0.27791693,
        -2.71614899,  0.9293593 ,  3.70744234, -2.5407848 ], requires_grad=True)
active_electrons = 4
active_orbitals = 6

H, qubits = qml.qchem.molecular_hamiltonian(symbols, x, charge=0, active_electrons = active_electrons, active_orbitals = active_orbitals)
singles, doubles = qchem.excitations(active_electrons, qubits)

hf_state = qchem.hf_state(active_electrons, qubits)



#compute the significant double-excitation gates:
def circuit_1(params, excitations):
    qml.BasisState(hf_state, wires=range(qubits))

    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(params[i], wires=excitation)
        else:
            qml.SingleExcitation(params[i], wires=excitation)
    return qml.expval(H)

dev = qml.device("default.qubit", wires=qubits)
cost_fn = qml.QNode(circuit_1, dev, interface="autograd")
circuit_gradient = qml.grad(cost_fn, argnum=0)
params = [0.0] * len(doubles)
grads = circuit_gradient(params, excitations=doubles)
print("Computed gradients for all possible Double Excitation Gates: \n")
for i in range(len(doubles)):
    print(f"Excitation : {doubles[i]}, Gradient: {grads[i]}")   
doubles_select = [doubles[i] for i in range(len(doubles)) if abs(grads[i]) > 1.0e-5]
print("")
print("Number of selected double-excitation gates: ", len(doubles_select))



# optimizing the parameters for the double-excitation gates for Ansatz-wavefunction construction
opt = qml.GradientDescentOptimizer(stepsize=0.5)
params_doubles = np.zeros(len(doubles_select), requires_grad=True)
for n in range(20):
    params_doubles = opt.step(cost_fn, params_doubles, excitations=doubles_select)
print("Done!")



#compute the significant single-excitation gates:
def circuit_2(params, excitations, gates_select, params_select):
    qml.BasisState(hf_state, wires=range(qubits))

    for i, gate in enumerate(gates_select):
        if len(gate) == 4:
            qml.DoubleExcitation(params_select[i], wires=gate)
        elif len(gate) == 2:
            qml.SingleExcitation(params_select[i], wires=gate)

    for i, gate in enumerate(excitations):
        if len(gate) == 4:
            qml.DoubleExcitation(params[i], wires=gate)
        elif len(gate) == 2:
            qml.SingleExcitation(params[i], wires=gate)
    return qml.expval(H)


cost_fn = qml.QNode(circuit_2, dev, interface="autograd")
circuit_gradient = qml.grad(cost_fn, argnum=0)
params = [0.0] * len(singles)
grads = circuit_gradient(
    params,
    excitations=singles,
    gates_select=doubles_select,
    params_select=params_doubles
)
print("Computed gradients for all possible Single Excitation Gates: \n")
for i in range(len(singles)):
    print(f"Excitation : {singles[i]}, Gradient: {grads[i]}")
singles_select = [singles[i] for i in range(len(singles)) if abs(grads[i]) > 1.0e-5]
print("")
print("Number of selected single-excitation gates: ", len(singles_select))

#Total Number of Gates selected to construct the Quantum Ansatz:
print("Total selected gates: "+  str(len(doubles_select) + len(singles_select)))

################# End code from the 'Building the adaptive circuit' tutorial ##################

################ Resume from the 'Optimization of molecular geometries' tutorial below: ################
hf = hf_state


#Construct optimized parametrized circuit that will be used to run the optimized VQE algorithm
# using the selected single and double excitation gates:
num_wires = qubits
dev = qml.device("lightning.qubit", wires=num_wires)
@qml.qnode(dev, interface="autograd")
def circuit(params, obs, wires):
    # prepares reference state
    qml.BasisState(hf, wires=wires)
    
    # apply all single excitations
    for i, singles in enumerate(singles_select):
        qml.SingleExcitation(params[i], wires=singles)
        
    # apply all double excitations
    for j, doubles in enumerate(doubles_select):
        qml.DoubleExcitation(params[j + len(singles_select)], wires=doubles)
                             
    # returns expectation value of the ansatz prepared from this quantum circuit:   
    return qml.expval(obs)

def cost(params, x):
    hamiltonian = H(x)
    return circuit(params, obs=hamiltonian, wires=range(num_wires)) 
    

# This function computes the finite-difference of the Hamiltonian itself with respect to each of the nuclear-coordinates
# itself, using a central-difference approximation.
def finite_diff(f, x, delta=0.001):
    """Compute the central-difference finite difference of a function"""
    gradient = []
    for i in range(len(x)):
        shift = np.zeros_like(x)
        shift[i] += 0.5 * delta
        res = (f(x + shift) - f(x - shift)) * delta**-1 # dH/dx            
        gradient.append(res)

    return gradient

def grad_x(params, x): #evaluate the expectation of the gradient components (each of the nuclear components of the Hamiltonian)
    grad_h = finite_diff(H, x)
    grad = [circuit(params, obs=obs, wires=range(num_wires)) for obs in grad_h]
    return np.array(grad)

# initialize optimizers:
# opt_theta = qml.GradientDescentOptimizer(stepsize=0.4)
# opt_x = qml.GradientDescentOptimizer(stepsize=0.001)
# opt_theta = qml.GradientDescentOptimizer(stepsize=0.2)
# opt_x = qml.GradientDescentOptimizer(stepsize=0.01)
opt_theta = qml.GradientDescentOptimizer(stepsize=0.2)
opt_x = qml.AdamOptimizer(stepsize=0.05)





# functions to keep track of convergence of O-H1 and O-H2 bond angle during water's geometry optimization
import math
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return (360-np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)) * 180/math.pi) #convert to degrees


# Re-initialize all relevant parameters to be optimized:
# nuclear-coordinate parameters:
symbols = ["Be", "H", "H"]
x = np.array([ 0.8765    ,  1.9857    , -2.4895    ,  0.83258936,  0.27791693,
        -2.71614899,  0.9293593 ,  3.70744234, -2.5407848 ], requires_grad=True)

# circuit parameters:
theta = np.array([0.0] * (len(doubles_select) + len(singles_select)), requires_grad=True)

print("Reinitialized gradient descent parameters!")




# re-define the hamiltonian needed to compute cost-function (otherwise it gives error: Hamiltonian function not callable
# when I run the actual gradient descent code below)
def H(x):
    return qml.qchem.molecular_hamiltonian(symbols, x, charge=0, active_electrons = active_electrons, active_orbitals = active_orbitals)[0]




# ~~~~~~~~~~~~~~ Final block: Calculating water's molecular geometry ~~~~~~~~~~~~~~~
from functools import partial

# store the values of the cost function
energy = []

# store the values of the first OH-bond length
bond_length = []

#store the value of the bond_angle:
bond_angle = []

# Factor to convert from Bohrs to Angstroms
bohr_angs = 0.529177210903

eps = 1e-05


# Calculate starting water-molecule parameters:
BeH1 = x[0:3] - x[3:6]
BeH2 = x[0:3] - x[6:9]
print("Starting bond length: " + str(np.linalg.norm(BeH1) * bohr_angs) + " Å")
print("Starting bond angle: " + str(angle_between(BeH2, BeH1)) + '\u00b0')
print("")

'''
Errors (and how to fix):

If error 'Hamiltonian object is not callable', re-run cell that defines H(x) all the way at the top.
If the starting bond length and angle are off, re-run cell that defines symbols and geometry for x all the way at the top
'''
start = time.time()
n = 0
while True:

    print("Starting step " + str(n) + ":")
    print("Calculating circuit parameters gradient")
    # Optimize the circuit parameters
    theta.requires_grad = True
    x.requires_grad = False
    theta, _ = opt_theta.step(cost, theta, x) # compare with final block of code in 'Adapt-VQE' tutorial on Pennylane

    print("Calculating nuclear coordinates parameters gradient")
    # Optimize the nuclear coordinates
    theta.requires_grad = False
    x.requires_grad = True
    
    grad_start = time.time()
    _, x = opt_x.step(cost, theta, x, grad_fn=grad_x)
    grad_end = time.time()
    
    print("Time it took to calculate gradient = ", grad_end-grad_start)
    print("Finished calculating nuclear coordinates parameters gradient\n")
    

    energy.append(cost(theta, x))
    bond_length.append(np.linalg.norm(x[0:3] - x[3:6]) * bohr_angs)
    
    #calculate bond-angle between both OH-bonds
    BeH1 = x[0:3] - x[3:6]
    BeH2 = x[0:3] - x[6:9]
    bond_angle.append(angle_between(BeH2, BeH1))

    if n % 1 == 0:
        print(f"Step = {n},  E = {energy[-1]:.8f} Ha,  bond length = {bond_length[-1]:.5f} Å, bond angle = {bond_angle[-1]:.5f}" + '\u00b0')

    n += 1
    if n <= 1:
        continue
    # Check maximum component of the nuclear gradient
    if np.max(grad_x(theta, x)) <= 1e-05 or np.abs(energy[-2]-energy[-1]) < eps:
        break
    print("")
end = time.time()

print("\n" f"Final value of the ground-state energy = {energy[-1]:.8f} Ha")
print("\n" "Ground-state equilibrium geometry")
print("%s %4s %8s %8s" % ("symbol", "x", "y", "z"))
for i, atom in enumerate(symbols):
    print(f"  {atom}    {x[3 * i]:.4f}   {x[3 * i + 1]:.4f}   {x[3 * i + 2]:.4f}")
    
print("")
print("Total time:", end-start, "seconds")

Computed gradients for all possible Double Excitation Gates: 

Excitation : [0, 1, 4, 5], Gradient: -0.05675774547928986
Excitation : [0, 1, 4, 7], Gradient: 0.0
Excitation : [0, 1, 4, 9], Gradient: 0.000741991486460103
Excitation : [0, 1, 4, 11], Gradient: 0.0
Excitation : [0, 1, 5, 6], Gradient: 0.0
Excitation : [0, 1, 5, 8], Gradient: -0.000741991486460103
Excitation : [0, 1, 5, 10], Gradient: 0.0
Excitation : [0, 1, 6, 7], Gradient: -0.05725210270187538
Excitation : [0, 1, 6, 9], Gradient: 0.0
Excitation : [0, 1, 6, 11], Gradient: 0.0
Excitation : [0, 1, 7, 8], Gradient: 0.0
Excitation : [0, 1, 7, 10], Gradient: 0.0
Excitation : [0, 1, 8, 9], Gradient: -0.07077033797425428
Excitation : [0, 1, 8, 11], Gradient: 0.0
Excitation : [0, 1, 9, 10], Gradient: 0.0
Excitation : [0, 1, 10, 11], Gradient: -0.05790194720065484
Excitation : [0, 2, 4, 6], Gradient: 0.0
Excitation : [0, 2, 4, 8], Gradient: 0.0
Excitation : [0, 2, 4, 10], Gradient: 0.005192956578481277
Excitation : [0, 2, 6, 8], Gr

Step = 11,  E = -15.43100030 Ha,  bond length = 1.05346 Å, bond angle = 211.10084°

Starting step 12:
Calculating circuit parameters gradient
Calculating nuclear coordinates parameters gradient
Time it took to calculate gradient =  318.85351061820984
Finished calculating nuclear coordinates parameters gradient
Step = 12,  E = -15.43631829 Ha,  bond length = 1.06441 Å, bond angle = 212.12870°

Starting step 13:
Calculating circuit parameters gradient
Calculating nuclear coordinates parameters gradient
Time it took to calculate gradient =  308.4198935031891
Finished calculating nuclear coordinates parameters gradient
Step = 13,  E = -15.44229678 Ha,  bond length = 1.07309 Å, bond angle = 213.05977°

Starting step 14:
Calculating circuit parameters gradient
Calculating nuclear coordinates parameters gradient
Time it took to calculate gradient =  330.9259190559387
Finished calculating nuclear coordinates parameters gradient
Step = 14,  E = -15.44886950 Ha,  bond length = 1.07974 Å, bond an

Calculating nuclear coordinates parameters gradient
Time it took to calculate gradient =  305.01295280456543
Finished calculating nuclear coordinates parameters gradient
Step = 38,  E = -15.52727900 Ha,  bond length = 1.19321 Å, bond angle = 222.81107°

Starting step 39:
Calculating circuit parameters gradient
Calculating nuclear coordinates parameters gradient
Time it took to calculate gradient =  308.41184520721436
Finished calculating nuclear coordinates parameters gradient
Step = 39,  E = -15.53264835 Ha,  bond length = 1.19835 Å, bond angle = 222.94335°

Starting step 40:
Calculating circuit parameters gradient
Calculating nuclear coordinates parameters gradient
Time it took to calculate gradient =  305.191086769104
Finished calculating nuclear coordinates parameters gradient
Step = 40,  E = -15.53705100 Ha,  bond length = 1.20282 Å, bond angle = 223.06148°

Starting step 41:
Calculating circuit parameters gradient
Calculating nuclear coordinates parameters gradient
Time it took t

Step = 64,  E = -15.56614875 Ha,  bond length = 1.21301 Å, bond angle = 222.07685°

Starting step 65:
Calculating circuit parameters gradient
Calculating nuclear coordinates parameters gradient
Time it took to calculate gradient =  305.7281816005707
Finished calculating nuclear coordinates parameters gradient
Step = 65,  E = -15.56685986 Ha,  bond length = 1.21521 Å, bond angle = 222.01952°

Starting step 66:
Calculating circuit parameters gradient
Calculating nuclear coordinates parameters gradient
Time it took to calculate gradient =  305.30529522895813
Finished calculating nuclear coordinates parameters gradient
Step = 66,  E = -15.56750350 Ha,  bond length = 1.21725 Å, bond angle = 221.95979°

Starting step 67:
Calculating circuit parameters gradient
Calculating nuclear coordinates parameters gradient
Time it took to calculate gradient =  309.9109790325165
Finished calculating nuclear coordinates parameters gradient
Step = 67,  E = -15.56809772 Ha,  bond length = 1.21915 Å, bond an


KeyboardInterrupt

